In [3]:
import sys
import os

# modify syspath directory to sys.path

package_path = os.path.abspath(os.path.join('..','core'))
if package_path not in sys.path:
    sys.path.append(package_path)

package_path = os.path.abspath(os.path.join('..','utils'))
if package_path not in sys.path:
    sys.path.append(package_path)

package_path = os.path.abspath(os.path.join('..'))
if package_path not in sys.path:
    sys.path.append(package_path)

import numpy as np
import qutip as qt
import math
import matplotlib.pyplot as plt

from IPython.display import HTML # both needed to disply gifs
from PIL import Image

# Import modules from the package
from core import create_hamiltonian as ch
from core import create_state as cs
from core import create_coherent_state
from core import time_evo
from core import time_evo_new
from core import time_evo_from_state
from core import time_evo_rd_ACL
from core import load_param
from core import load_H_list
from core import load_result
from core import load_tlist
from core import operators
from core.schmidt_solve import compute_schmidt_states_all_time
from core.eigen_ener_states import eigen_ener_states as eig
from core import annihilation_operator
from utils.entropy import compute_VN, plot_VN, compute_VN_time, plot_VN_numpy
from utils import collapse_gif
from utils.collapse_gif import collapse_gif
from utils.display_gif import display_gif
from utils.distribution_gif import gif_distribution_eig_total
from utils.distribution_gif import prob_gif
from utils.distribution_gif import gif_distrib_H_s_int
from utils.distribution_gif import gif_distrib_system_first_eig
from utils.plot import plot_basic
from utils.plot import plot_with_binning
from utils.basic_functions import state_distribution_eig_tot
from utils.basic_functions import random_dephasing
from utils.basic_functions import get_state_probabilities
from utils.basic_functions import position
from utils.basic_functions import copy_cat
from utils.basic_functions import plot_einselection
from utils.overlap import overlap
from utils.overlap import gif_schmidt_overlap
from utils.interference import interference_plot
from utils.expectations import exp_val_time
from utils.plot_hamiltonians import plot_hamiltonians

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.titlesize'] = 16             # Title font size
plt.rcParams['axes.titleweight'] = 'bold'       # Title font weight
plt.rcParams['axes.labelsize'] = 14             # X and Y label font size
plt.rcParams['axes.labelweight'] = 'bold'       # X and Y label font weight
#plt.rcParams['axes.grid'] = True                # Enable grid
#plt.rcParams['grid.alpha'] = 0.7                # Grid transparency
#plt.rcParams['grid.linestyle'] = '--'           # Grid line style
#plt.rcParams['grid.color'] = 'gray'             # Grid color

# Tick settings
plt.rcParams['xtick.labelsize'] = 12            # X tick label size
plt.rcParams['ytick.labelsize'] = 12            # Y tick label size
plt.rcParams['xtick.direction'] = 'in'          # X tick direction
plt.rcParams['ytick.direction'] = 'in'          # Y tick direction
plt.rcParams['xtick.major.size'] = 6            # X major tick size
plt.rcParams['ytick.major.size'] = 6            # Y major tick size

# Legend settings
plt.rcParams['legend.fontsize'] = 12            # Legend font size
plt.rcParams['legend.frameon'] = True           # Enable legend frame
plt.rcParams['legend.framealpha'] = 0.9         # Legend frame transparency
plt.rcParams['legend.loc'] = 'best'             # Legend location

# Line and marker settings
plt.rcParams['lines.linewidth'] = 2             # Line width
plt.rcParams['lines.markersize'] = 6            # Marker size

custom_colors = ['#1c4587', '#e6194B', '#2ca02c', '#d62728', '#9467bd', 
                 '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=custom_colors)

Decoherence and 2 locality:

Here we look at the effect of the environment Hamiltonian being 2-local on decoherence. We know that decoherence happens very well with 2-local Hamiltonians as we see its effects in the classical world around us and every time we work with quantum systems. The question is then not "are 2-local Hamiltonians capable of decoherence., nor is it "Can we achieve deocherence without 2-locality" as a fully random matriux is more scrambling than a 2-local one. The question is "What does the constraint of 2-locality do to the deocherence we deal with everyday?"

As a bonus we can also look at the impact of geometric locality on decoherence. ie, make the spin chain a chain and only the nearest qubit interact with it and then the qubits themselfs only interact with nn.

potential result: the environment interactions are 2-local only wrt a basis. does that mean we should expect something? ie, deochernece speed shouldn't depend on the basis in which we are looking at thing? Or rather, somehow decoherence and the einselected pointer basis is more readily a basis in which the environment looks 2-local?

What needs to be done: Define the time evol and Hamiltonian creation functions that are 2-local.

Do simulations:

Define appropriate decoherence quantities to look at.

Make some pretty graphs.

Initial choices: Work with simplest possible Hamiltonian


We create the simplest decoherence model, so a simple HsxHe...

H=h1x(sum_ihisixsi)

need to show it but i think that I can make a complet basis of hermitian operators in 2qubit space with XX,XZ,YY,ZX,ZZ. not i didn't include the opposities, ie XZ but not ZX



<h1>Generate Hamiltonian

In [27]:
ar=np.array
kr=np.kron
idn=np.identity
sx=ar([[0,1],[1,0]])
sy=ar([[0,-1j],[1j,0]])
sz=ar([[1,0],[0,-1]])
id2=ar([[1,0],[0,1]])

n=10

H = np.zeros((2**n, 2**n), dtype=np.complex64)

for i in range(n):
    for j in range(n):
        if j>i:
            s1=kr(kr(idn(2**i),sx),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sx),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2
            s1=kr(kr(idn(2**i),sx),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sz),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2
            s1=kr(kr(idn(2**i),sz),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sx),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2
            s1=kr(kr(idn(2**i),sy),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sy),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2
            s1=kr(kr(idn(2**i),sz),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sx),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2
            s1=kr(kr(idn(2**i),sx),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sz),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2
            s1=kr(kr(idn(2**i),sz),idn(2**(n-i-1)))
            s2=kr(kr(idn(2**j),sz),idn(2**(n-j-1)))
            H+=np.random.rand()*s1@s2

#plt.imshow(H, cmap='hot', interpolation='nearest')
#plt.colorbar()  # Adds a color bar to interpret the scale
#plt.title("Heat Map of Matrix H")
#plt.show()

<h1>Create a state

In [41]:
n_system_qubits = 1  # Number of qubits in the system
n_environment_qubits = n  # Number of qubits in the environment

# Create the superposition state for the system
system_superposition_state = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
random_state = qt.rand_ket(2**n)
state = qt.tensor(system_superposition_state, random_state)
print(f"Dimension of the combined state: {state.dims}")

Dimension of the combined state: [[2, 1024], [1, 1]]


<h1>Do time evolution

In [42]:
def time_evo(H_e,state,log=0,tmax=10,ind_nb=100,file_name="default"):
    sz=ar([[1,0],[0,-1]])
    H_e=qt.Qobj(H_e)
    H_s = qt.Qobj(sz)  # Identity for system
    # Total Hamiltonian: Tensor product of system identity and environment Hamiltonian
    H = qt.tensor(H_s, H_e)

    tlist = np.linspace(0, tmax, ind_nb) # Linear spacing
    if log == 0:
        tlist = np.linspace(0, tmax, ind_nb)  # Linear spacing
    elif log == 1:
        tlist = np.logspace(np.log10(1), np.log10(tmax+1), ind_nb)-1  # Logarithmic spacing
    else:
        raise ValueError("Invalid value for 'log'. It should be either 0 or 1.")
    info_list=[tmax, ind_nb,log,tlist]
    
    # Perform time evolution of the combined system
    result = qt.mesolve(H, state, tlist, [], [])

    # Save outputs in a .txt file
    outputs_dir = "outputs/simulation_results"
    if not os.path.exists(outputs_dir):
        os.makedirs(outputs_dir)
    
    # Save parameters in a .txt file
    params_file_path = os.path.join(outputs_dir, "params_" + file_name)
    with open(params_file_path, "w") as f:
        tmax, ind_nb,log
        f.write(f"tmax === {tmax}\n")
        f.write(f"ind_nb === {ind_nb}\n")
        f.write(f"log === {log}\n")
    
    # Save parameters in a .txt file
    tlist_file_path = os.path.join(outputs_dir, "tlist_" + file_name)
    np.save(tlist_file_path, tlist)
    
    # Save result in a .txt file
    result_file_path = os.path.join(outputs_dir, "result_" + file_name)
    qt.qsave(result, result_file_path)
    
    # Save H_list in a .txt file
    
    H_path = os.path.join(outputs_dir, "H_list_" + file_name)
    qt.qsave(H, H_path)
    
    return result, tlist, H, state, info_list

In [43]:
result, tlist, H, state, info_list = time_evo(H,state,log=0,tmax=10,ind_nb=100,file_name="2_locality_first_attempt")

In [40]:
import qutip as qt

n_system_qubits = 1  # Number of qubits in the system
n_environment_qubits = n  # Number of qubits in the environment

# Create the superposition state for the system
system_superposition_state = (qt.basis(2, 0) + qt.basis(2, 1)).unit()

# Create the product state for the environment (all qubits in the 0 state)
environment_state = qt.tensor([qt.basis(2, 0) for _ in range(n_environment_qubits)])

# Combine the system and environment states
state = qt.tensor(system_superposition_state, environment_state)

# Reshape the combined state to have dimension [2, n]
combined_state = state.reshape([2, 2**n_environment_qubits])

print(f"Dimension of the combined state: {combined_state.dims}")


AttributeError: 'Qobj' object has no attribute 'reshape'